## <font color=#0099CC>mIAx - Taller Renta Fija - ANÁLISIS CARTERA DE RENTA FIJA</font>

En esta práctica, desarrollaremos un análisis relativamente exahustivo de un universo de Renta Fija, en concreto, de bonos corporativos. Además, construiremos y analizaremos varias carteras.

Para ello, contaremos con la siguiente información almacenada en la carpeta *data*:
- Universo de bonos, con sus características esenciales (fichero *universo.csv*)
- Histórico de precios de cierre del universo de bonos anterior (fichero *precios_historicos_universo.csv*)
- Curva de tipos de interés €STR (fichero *curvaESTR.csv*)
- Histórico de precios de otros índices que nos serán de utilidad (fichero *precios_historicos_varios*):
    - Índices de crédito: ITRAXX Main y ITRAXX XOVER. Serán útiles para la cobertura del riesgo de crédito.
    - Futuros sobre el *Schatz* (DU1), *BOBL* (OE1) y *BUND* (RX1). Serán útiles para la cobertura de los tipos de interés.
    - Índice de crédito *RECMTREU*, que valdría como benchmark de las carteras que construyamos.

No necesariamente se usará toda toda la información

En líneas generales, estos son los ejercicios que completaremos, aunque los detallaremos más en cada apartado:
1. Análisis de datos. En esta sección, haremos un análisis de la información que tenemos de cada bono y lo que significa. Asimismo, haremos los tratamientos y limpieza que necesitemos para luego poder usarlos.
2. Valoración de los bonos del universo utilizando la curva de descuento y bajo ciertas asunciones. Comparación de estos precios con los precios de mercado.
3. Cálculo del spread que pagan los bonos sobre la curva.
4. Cálculo de *yield*, duración y convexidad.
5. Contrucción de una cartera equiponderada con todos los bonos del universo. Contraste con el benchmark (os proponemos el índice RECMTREU para el que os hemos dado los precios) y backtest de la estrategia. ¡OJO! El índice es *Total Return*.
6. Tienes el mandato de construir una cartera de como máximo **20** bonos corporativos con ese universo y una serie de restricciones y, claro, maximizando la rentabilidad total de la cartera:
    - La duración de la cartera no debe superar los 3 años
    - La exposición a emisiones HY no puede superar el 10% de la cartera
    - No puedes invertir en deuda subordinada
    - No se puede invertir en emisiones de tamaño igual o inferior a 500 millones
    - No se puede invertir más de un 10% del capital en una misma emisión
    - No puede haber más de un 15% de concentración en un mismo emisor
    (¡OJO! No estamos teniendo en cuenta en este ejercicio si hubiera un mínimo de inversión, lo cuál sería un dato relevante tener en cuenta en un caso real)

    6.1. Constrúyela a fecha de hoy

    6.2. Teniendo en cuenta la naturaleza que nos están pidiendo para la cartera, ¿añadirías alguna otra restricción?

    6.3. ¿Cómo medirías el riesgo de crédito de la cartera?

    6.4. ¿Cómo medirías el riesgo de liquidez de la cartera?

    6.5. Describe cómo habría que hacer el backtest de esta cartera
7. Ahora, se te pide que cubras la exposición de la cartera a los tipos de interés. Con la información que tienes, ¿cómo lo harías?
8. ¿Y si quisieras cubrir total o parcialmente el riesgo de crédito? Usa de nuevo la información que tienes.
9. ¿Cómo construirías tu cartera? ¿Se te ocurre alguna estrategia específica, por ejemplo, de valor relativo?


📣 <font color=#CC6600>**¡NORMAS!**</font>

La práctica se puede hacer en grupos de hasta **3 personas** y deberá entregarse antes del **27 de noviembre**.

Cada grupo expondrá una parte de los ejercicios en la clase del día 27, donde la resolveremos juntos a modo de taller. Esta exposición contará hasta **1 punto** de la nota final.

Además, se valorará positivamente para la práctica la participación en las clases.

📣 <font color=#CC6600>**¡IMPORTANTE!**</font>

Todo el código implementado debe estar debidamente comentado e incluir conclusiones de los resultados obtenidos para optar a la máxima puntuación. Asimismo, se debe responder a las preguntas planteadas.

Las propuestas de mejora o posibles trabajos futuros se valorarán positivamente también.

Usa las fuentes de información que consideres necesarias para apoyar tus respuestas.

### <font color=#336699>Librerías</font>

In [ ]:
from importlib import resources
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

In [ ]:
# Estilo
plt.style.use('dark_background')

In [ ]:
# Fecha de análisis
fecha_analisis = pd.to_datetime('2025-10-01')

### <font color=#336699>1. Datos</font>

<style>.gray {background-color: #595959}

</style><div class="gray">❕💬¿Qué observas en los datos? Analiza la información que tenemos del universo</div>

Haz un primer análisis visual de la información que tenemos del universo de bono.
Entre otras, plantéate cuestiones como:
- ¿Divisas?
- ¿Tipo de bonos? ¿Fijo/Flotante? ¿Prelación? ¿Opcionalidad? ¿Hay bonos perpetuos?
- ¿Sectores? ¿Emisores? Si invirtéramos en todos los bonos, ¿dirías a priori que la cartera está diversificada?
- ¿Ratings? (Riesgo de crédito)
- ¿Otros datos cuantitativos?
    - Riesgo de liquidez - Horquillas y nominal vivo
- ¿Hay *gaps* en la información que vamos a tener que tratar?

In [ ]:
from pathlib import Path
# Ruta relativa desde src/ hacia data/ (un nivel arriba)
# Si el notebook se ejecuta desde src/, esta ruta funcionará
data_path = Path('../data/universo.csv')
universo = pd.read_csv(data_path, sep=';', encoding='utf-8-sig')

#### <font color=#808080>Divisas</font>

In [ ]:
print("ANÁLISIS DE DIVISAS:")
currency = universo['Ccy'].value_counts(dropna = False) 
for curr, quantity in currency.items():
    print(f"  - {curr}: {quantity} bonos")

**Conclusión**: Todos los bonos están denominados en EUR. Esto elimina el riesgo por cambio de divisas.

In [ ]:
# Análisis de tipos de bonos
print(" TIPOS DE BONOS:")

# Tipo de cupón
print("Tipo de Cupón:")
coupon_types = universo['Coupon Type'].value_counts(dropna = False) 
for coupon, quantity in coupon_types.items():
    print(f"  - {coupon}: {quantity} bonos")

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(14, 10))

# Gráfico 1: Tipo de cupón
coupon_types.plot(kind='pie', ax=axes[0,0], autopct='%1.1f%%', colors=['#4CAF50', '#FF9800'])
axes[0,0].set_title('Distribución por Tipo de Cupón')
axes[0,0].set_ylabel('')

# Prelación
print("Prelación (Seniority):")
seniority = universo['Seniority'].value_counts(dropna = False) 
for senior, quantity in seniority.items():
    print(f"  - {senior}: {quantity} bonos")

# Gráfico 2: Prelación
seniority.plot(kind='barh', ax=axes[0,1], color='#2196F3')
axes[0,1].set_title('Tipos de Prelación')
axes[0,1].set_xlabel('Número de Bonos')

# Callable
print("Optionalidad (Callable):")
callable_analysis = universo['Callable'].value_counts(dropna = False) 

for call , quantity in callable_analysis.items():
    print(f"  - {call}: {quantity} bonos")

callable_pct = callable_analysis.get('Y', 0)/len(universo)*100
print(f"-> {callable_pct:.1f}% de los bonos son callable")

# Gráfico 3: Callable
callable_analysis.plot(kind='pie', ax=axes[0,2], autopct='%1.1f%%',
                       colors=['#E91E63', '#9C27B0'],
                       labels=['Callable', 'No Callable'])
axes[0,2].set_title('Bonos Callable vs No Callable')
axes[0,2].set_ylabel('')

# Frecuencia de cupón
print("Frecuencia de Cupón:")
freq = universo['Coupon Frequency'].value_counts(dropna = False) 
for frequency, quantity in freq.items(): #Cambio el print para que muestre la información más clara
    print(f"  - {frequency} pago(s) por año: {quantity} bonos")

# Gráfico 4: Frecuencia
freq.plot(kind='bar', ax=axes[1,0], color='#00BCD4')
axes[1,0].set_title('Frecuencia de Pago de Cupón')
axes[1,0].set_xlabel('Pagos por año')
axes[1,0].set_ylabel('Número de Bonos')
axes[1,0].tick_params(axis='x', rotation=0)


# Maturity:
print("Bonos perpetuos o no (Maturity):")

maturity_date = universo[universo['Maturity'].notna() & (universo['Maturity'] != '')]
no_maturity_date= universo[universo['Maturity'].isna() | (universo['Maturity'] == '')]

print(f"  - Bonos con fecha de vencimiento: {len(maturity_date)}")
print(f"  - Bonos perpetuos (sin fecha): {len(no_maturity_date)}")

print("Para valorar los bonos perpetuos hay que tener en cuenta que la fecha de vencimiento es la de la primera call.")

# Gráfico 4: Maturity
labels = ['Con fecha de vencimiento', 'Perpetuos (sin fecha)']
sizes = [len(maturity_date), len(no_maturity_date)]
colors = ['#4CAF50', '#FFC107']

axes[1,1].pie(sizes, colors=colors, labels=labels, autopct='%1.1f%%', startangle=90)
axes[1,1].set_title('Distribución de Bonos por Maturity')

# Ocultar el grafico vacio
axes[1,2].axis('off')

plt.tight_layout()
plt.show()

#### <font color=#808080>Sectores y emisores</font>

In [ ]:
# Análisis de sectores y emisores
print("ANÁLISIS DE SECTORES Y EMISORES:")


# Sectores
sector_counts = universo['Industry Sector'].value_counts(dropna = False) 
print("Distribución por Sectores:")
for sector, quantity in sector_counts.items():
    print(f"  - {sector} : {quantity} bonos")
print(f"Total de sectores únicos: {universo['Industry Sector'].nunique()}")

# Emisores
emisor_counts = universo['Issuer'].value_counts(dropna = False) 
print(f"Total de emisores únicos: {universo['Issuer'].nunique()}")

print("Top 10 Emisores:")
top_emisors = emisor_counts.head(10)

for emisor, quantity in top_emisors.items():
    print(f"  - {emisor}: {quantity} bonos")

# Concentración
top_5_emisores_pct = top_emisors.head(5).sum() / len(universo) * 100
top_5_sectores_pct = sector_counts.head(5).sum() / len(universo) * 100

print("Análisis de Concentración:")
print(f"  - Top 5 emisores: {top_5_emisores_pct:.1f}% del universo")
print(f"  - Top 5 sectores: {top_5_sectores_pct:.1f}% del universo")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de sectores
sector_counts.plot(kind='barh', ax=axes[0], color=plt.cm.viridis(np.linspace(0, 1, len(sector_counts))))
axes[0].set_title('Distribución por Sectores')
axes[0].set_xlabel('Número de Bonos')

# Gráfico de concentración de emisores
emisor_counts.head(15).plot(kind='barh', ax=axes[1], color='#FF5722')
axes[1].set_title('Top 15 Emisores por Número de Bonos')
axes[1].set_xlabel('Número de Bonos')

plt.tight_layout()
plt.show()

**Conclusión**: Sí que sería una cartera diversificada, si se inviertera en todos los bonos. Ya que los emiten 613 empresas distintas de 9 sectores diferentes.

#### <font color=#808080>Ratings (Riesgo de crédito)</font>

In [ ]:
## Aquí tengo dudas. !!!Repasar , porque no sé si son las distintas agencias dan un raiting distinto, pero que luego se llama igual, o está todo rateado por la misma agencia?

print("ANÁLISIS DE RATINGS:")

rating_order = [
    'AAA+', 'AAA', 'AAA-', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-',
    'BBB+', 'BBB', 'BBB-', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-',
    'CCC+', 'CCC', 'CCC-', 'CC', 'C', 'D', 'NR'
]

# Distribución de ratings
print("\n• Distribución de Ratings Específicos:")
rating_dist = universo['Rating'].value_counts(dropna = False) 
#rating_dist = rating_dist.reindex(rating_order).dropna()
for rating, quantity in rating_dist.items():
    print(f"  - {rating}: {int(quantity)} bonos")


print("\n• Resumen:")

rating_category_map = {}
for rating in rating_order:
    if rating == 'NR':
        rating_category_map[rating] = 'NR'
    elif rating_order.index(rating) <= rating_order.index('BBB-'):
        rating_category_map[rating] = 'IG'
    else:
        rating_category_map[rating] = 'HY'

# Mapear categorías desde rating_dist
category_counts = {'IG': 0, 'HY': 0, 'NR': 0}
for rating, count in rating_dist.items():
    categoria = rating_category_map.get(rating, 'NR')
    category_counts[categoria] += count

# Mostrar resumen
total_bonos = rating_dist.sum()

for cat in ['IG', 'HY', 'NR']:
    count = category_counts[cat]
    pct = count / total_bonos * 100
    print(f"  • {cat}: {int(count)} bonos ({pct:.1f}%)")

print("\n")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart de categorías (IG, HY, NR)
category = ['IG', 'HY', 'NR']
porcentajes = [category_counts[cat] / total_bonos * 100 for cat in category]
axes[0].pie(
    porcentajes,
    labels=[f'{cat} - {pct:.1f}%' for cat, pct in zip(category, porcentajes)],
    autopct='%1.1f%%',
    startangle=90,
    colors=['#4CAF50', '#FF9800', '#9E9E9E']
)
axes[0].set_title('Distribución por Categoría de Rating')
axes[0].set_ylabel('')

# Bar chart de ratings específicos ordenados
rating_dist.plot(kind='bar', ax=axes[1], color='#3F51B5')
axes[1].set_title('Ratings Específicos (ordenados)')
axes[1].set_xlabel('Rating')
axes[1].set_ylabel('Número de Bonos')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


**Conclusión:** Hay un bono que tiene valor rating #N/A N/A. Consideramos que se trata un error de los datos ya que si no tuviese rating sería NR.

#### <font color=#808080>Riesgo de liquidez - Horquillas y nominal vivo</font>

In [ ]:
print("\n• Riesgo de liquidez:")

hor = universo[universo['Ask Price'] - universo['Bid Price'] > 0.5]

print(f"Los bonos que tienen riesgo de liquidez son: {len(hor)}")
hor.head(5)

In [ ]:
print("\n• Nominal vivo (Outstanding Amount):")
print("\n Los bonos con mayor nominal vivo suelen tener más negociación. Top 5: ")
oa = universo[["ISIN", "Issuer", "Outstanding Amount"]].sort_values(by="Outstanding Amount", ascending=False)
oa.head(5)

In [ ]:
print("\n Top 5 bonos con menor nominal vivo: ")
oa.tail(5)

#### <font color=#808080>Resto de información</font>

Otro datos que consideramos que se pueden analizar son PD 1YR.

In [ ]:
pd_1y = universo[['ISIN', 'Issuer', 'Rating', 'PD 1YR']]

pd_1y.sort_values(by="PD 1YR", ascending=False).head()

Destacamos que en el top cinco de bonos con mayor probabilidad de impago a un año, no aparece ninguno que no tenga rating.

In [ ]:
nr_pd_1y = universo[universo['Rating'] == 'NR'][['ISIN', 'Issuer', 'Rating', 'PD 1YR']]
nr_pd_1y.sort_values(by="PD 1YR", ascending=False).head()

In [ ]:
universo.isna().sum()

**GAPs de información:** 
Podemos ver que hay 19 bonos que no tienen Maturity ni Penultimate Coupon Date, son bonos perpetuos.
Hay 24 bonos que no tiene PD 1YR, no tenemos esa información. 
También podemos comprobar que de los 615 bonos que no tienen Next Call Date ninguno es callable.
Hay un bono que no tiene rating. Dada su baja probabilidad, 0,000436, estaría en un rating AAA+. Sin embargo, consideramos que no es bono válido. 

In [ ]:
len(universo[universo['Next Call Date'].isna() & universo['Callable'] == 'Y']['ISIN'])

In [ ]:

# Función para limpiar columnas numéricas
def clean_numeric_column(col):
    """Limpia columnas numéricas reemplazando valores no válidos"""
    if col.dtype == object:
        # Reemplazar comas por puntos como separador decimal si es necesario
        col = col.astype(str).str.replace(',', '.')
        # Reemplazar cadenas de texto comunes por NaN
        col = col.replace(['#N/D', '#N/A', 'N/A'], np.nan)
        # Convertir a numérico, forzando los errores a NaN
        col = pd.to_numeric(col, errors='coerce')
    return col

# Fecha de análisis
fecha_analisis = pd.to_datetime('2025-10-01')
print(f"Fecha de análisis: {fecha_analisis.strftime('%Y-%m-%d')}")


# Limpiar columnas numéricas
numeric_columns = ['Price', 'Coupon', 'PD 1YR', 'Outstanding Amount',
                   'Coupon Frequency', 'Bid Price', 'Ask Price']
for col in numeric_columns:
    if col in universo.columns:
        universo[col] = clean_numeric_column(universo[col])

# Convertir fechas
date_columns = ['Maturity', 'Next Call Date', 'First Coupon Date',
                'Penultimate Coupon Date', 'Issue date']
for columna in date_columns:
    if columna in universo.columns:
        universo[columna] = pd.to_datetime(universo[columna], format='%d/%m/%Y', errors='coerce')

# Calcular años hasta vencimiento
universo['years_to_maturity'] = (universo['Maturity'] - fecha_analisis).dt.days / 365.25

print(f"\n nº fondos: {len(universo)} bonos")
print(" Información general del universo:")
print(universo.head())

universo.info()

En esta sección, analizamos el resto de ficheros para ver qué información tenemos y, en caso de haber *gaps*, limpiar los datos antes de trabajar con ellos.

##### <font color=#CC6600>Precios bonos universo</font>

In [ ]:
precios_hist_universo_path = Path('../data/precios_historicos_universo.csv')
precios_historicos_universo = pd.read_csv(precios_hist_universo_path, sep=';', encoding='utf-8-sig', header=None)

In [ ]:
precios = precios_historicos_universo.drop(precios_historicos_universo.columns[1], axis=1)

Se elimina la columna con fecha, 2023-10-02, ya que es domingo y no se tiene datos anteriores.

In [ ]:
precios.iloc[0,0] = 'ISIN'
fechas = precios.iloc[0, 1:]
columna_isin = precios.iloc[1:, 0]

In [ ]:
precios_nd = precios.iloc[1:,1:].copy()
precios_nd.replace('#N/D', pd.NA, inplace = True)
precios_nan = precios_nd.apply(pd.to_numeric, errors='coerce')

precios_ffill = precios_nan.ffill(axis=1)

Se sustituye #N/D por NaN, y se rellenan los datos con los precios del dia anterior en caso de que sea fin de semana. 

In [ ]:
fila_fechas = pd.DataFrame([['ISIN'] + list(fechas)], columns=precios.columns)
data = pd.concat([columna_isin.reset_index(drop=True), precios_ffill.reset_index(drop=True)], axis=1)
data.columns = precios.columns

In [ ]:
final_precios_hist_univ = pd.concat([fila_fechas, data], ignore_index=True)

##### <font color=#CC6600>Otros precios</font>

Para terminar con el análisis de datos, falta lo leído en los ficheros de *"precios_historicos_varios.csv"* y *curvaESTR.csv*.

In [ ]:
precios_hist_varios_path = Path('../data/precios_historicos_varios.csv')
precios_historicos_varios = pd.read_csv(precios_hist_varios_path, sep=';', encoding='utf-8-sig')

In [ ]:
precios_historicos_varios.columns.values[0] = 'Fecha'
precios_historicos_varios.head()

En el archivo de precios_historicos_varios, se encuentra los datos de cinco indices con el rango de fechas hasta 2023-10-02. Los indices son:  
* ITRX EUR CDSI GEN 5Y Corp,  iTraxx Europe Main 5Y:  representa un índice de Credit Default Swaps (CDS) sobre 125 empresas europeas con calificación de crédito investment grade, con horizonte en 5 años. 
* ITRX XOVER CDSI GEN 5Y Corp: Índice de Credit Default Swaps (CDS) con horizonte en 5 años, 75 empresas europeas con calificación por debajo de grado de inversión, high yield.
* DU1 Comdty, Euro-Bund Futures: su subyacente es el bono del gobierno alemán con vencimiento a 10 años.
* OE1 Comdty, Euro-OAT Futures: su subyacente es el bono del gobierno francés y con vencimiento a 5 años.
* RX1 Comdty, Euro-Buxl Futures: su subyacente es el bono del gobierno aleman con vencimiente de 24 1a 35 años. 
* RECMTREU Index, Bloomberg MSCI EUçuro Corporate SRI Total Return Index Unhedged: Indice de renta fija corporativa.

In [645]:
curva_estr_path = Path('../data/curvaESTR.csv')
curva_estr = pd.read_csv(curva_estr_path, sep=';', encoding='utf-8-sig')

In [ ]:
#curva_estr = curva_estr.drop(index=0)


In [646]:
curva_estr.head()

,Date,Market Rate,Zero Rate,Discount
0,01/10/2025,NaN,NaN,1.000000
1,10/10/2025,1.927,1.95350,0.999518
2,17/10/2025,1.928,1.95402,0.999144
3,03/11/2025,1.928,1.95320,0.998236
4,03/12/2025,1.929,1.95264,0.996635


In [647]:
#Convertir fechas a días desde hoy
curva_estr['Date'] = pd.to_datetime(curva_estr['Date'], dayfirst=True)
fecha_valor = pd.to_datetime('10/10/2025', dayfirst=True)
curva_estr['Days'] = (curva_estr['Date'] - fecha_valor).dt.days
curva_estr['YearFrac'] = curva_estr['Days'] / 365  # o 365 si usas ACT/365


In [649]:
#- La tasa forward entre los puntos t_1 y t_2, expresada en términos continuo

curva_estr['ForwardRate'] = np.nan
for i in range(1, len(curva_estr)):
    t1 = curva_estr.at[i-1, 'YearFrac']
    t2 = curva_estr.at[i, 'YearFrac']
    D1 = curva_estr.at[i-1, 'Discount']
    D2 = curva_estr.at[i, 'Discount']
    if pd.notna(D1) and pd.notna(D2) and t2 > t1:
        curva_estr.at[i, 'ForwardRate'] = np.log(D1 / D2) / (t2 - t1)



### <font color=#336699>2. Valoración</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 En esta sección, valoraremos los bonos utilizando la curva. Para ello, crea una función (puedes hacerlo en un .py aparte) que con las **características del bono, la curva y un spread de crédito** devuelva la valoración del bono (incluyendo **precio limpio, cupón corrido y precio sucio**).

Si asumimos que el **spread de crédito es 0**, y la ejecutamos para el 01/10/2025...
- ¿Qué observas si comparas los precios obtenidos y los precios de mercado?
- ¿Crees que la diferencia se debe a un factor relacionado sólo con el riesgo crediticio?
- ¿Qué otros factores influyen en ese spread?

Para la valoración, haz las siguientes simplificaciones:

- Asume que el vencimiento de los bonos perpetuos (para los que no hay vencimiento) es la próxima fecha call.
- Asume que todos aquellos bonos que tengan call serán calleados. Por lo tanto, usa la fecha call como fecha de vencimiento.
- Asume que los cupones son fijos hasta vencimiento (aunque alguno cambie a lo largo de la vida del bono).
- Usa la base de cálculo ACT/365. No tengas en cuenta la convención de día hábil.

Ten en cuenta que necesitarás una función de interpolación también. Interpola los factores de descuento exponencialmente.

</div>

In [ ]:
# Establecemos la fecha
fecha_valor = datetime.datetime(2025,10,1)

In [ ]:
import simulation

curva = simulation.read_curve('curvaESTR.csv')
simulation.plot_curve(curva)
# Primer elemento
#simulation.evaluate_bond(fecha_valor, universo.iloc[0], curva, 0)


### <font color=#336699>3. Spread</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Calculemos ahora los spreads que debemos añadir a la curva con un movimiento paralelo para que cuadren los precios de mercado que tenemos. Para ello, usa la función de valoración del apartado anterior.
- ¿Qué observas? ¿Tienen sentido los resultados?
- ¿Con qué datos de los que tenemos compararías para ver si los resultados son coherentes?

</div>

### <font color=#336699>4. YTM, Duración, Convexidad</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Calculemos ahora la siguiente información, también relacionada con la rentabilidad y riesgo de las emisiones:
- *Yield* - Por simplicidad, en este caso, en el caso de los bonos callable, nos quedaremos con la fecha call, como en el ejercicio anterior. Usa las mismas asunciones que para la valoración y el spread.
- Duración
- Convexidad

Responde a las siguientes preguntas:
- ¿Que relación hay entre la TIR calculada y el spread calculado en el apartado anterior?
- ¿Qué relación hay entre la duración y el vencimiento? ¿Qué refleja la duración? ¿De qué otra forma se podría obtener esta sensibilidad?
- Estima el precio del bono usando la duración y convexidad, ¿qué observas?

</div>

### <font color=#336699>5. Cartera equiponderada</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Crea un algoritmo de inversión que consista en una cartera equiponderada, asignando el mismo peso a todos los bonos vivos en cada fecha de rebalanceo.

Asume rebalanceo mensual, y que no tenemos costes más allá de los implícitos en el propio precio, calcula la evolución que hubiese tenido tu algoritmo. Por simplificación, utiliza los precios MID que se te dan.

Asumiendo que el benchmark de la cartera es el índice que se nos da: *RECMTREU Index*. Contrasta la evolución de tú cartera contra dicho benchmark. Ten cuidado porque es un índice *Total Return*.

- ¿Qué sería lo más correcto en lugar de utilizar los precios MID?
- ¿Se te ocurre algún otro benchmark que se podría utilizar?

</div>

### <font color=#336699>6. Cartera mandato</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Como adelantábamos en el enunciado, tienes el mandato de construir una cartera de como máximo **20** bonos corporativos con ese universo y una serie de restricciones y, claro, maximizando la rentabilidad total de la cartera:
- La duración de la cartera no debe superar los 3 años
- La exposición a emisiones HY no puede superar el 10% de la cartera
- No puedes invertir en deuda subordinada
- No se puede invertir en emisiones de tamaño igual o inferior a 500 millones
- No se puede invertir más de un 10% del capital en una misma emisión
- No puede haber más de un 15% de concentración en un mismo emisor
(¡OJO! No estamos teniendo en cuenta en este ejercicio si hubiera un mínimo de inversión, lo cuál sería un dato relevante tener en cuenta en un caso real)

1. Teniendo en cuenta la naturaleza que nos están pidiendo para la cartera, ¿añadirías alguna otra restricción?

2. ¿Cómo medirías el riesgo de crédito de la cartera?

3. ¿Cómo medirías el riesgo de liquidez de la cartera? ¿Se te ocurre alguna otra información que se podría utilizar aunque no se te haya dado?

4. Describe cómo habría que hacer el backtest de esta cartera, no hace falta que lo implementes en este caso

</div>

##### <font color=#CC6600>Riesgo de crédito</font>

##### <font color=#CC6600>Riesgo de liquidez</font>

##### <font color=#CC6600>Backtest</font>

### <font color=#336699>7. Cobertura tipos de interés</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Utiliza alguno de los siguientes instrumentos de los que te hemos dado para cubrir la duración (sensibilidad de tipos de interés) de la cartera que has construido según el mandato. Asume una inversión en la cartera de 10 millones:

- Futuros sobre el *Schatz* (ticker: DU1) - Duración a 01/10/2025: 1.92
- Futuros sobre el *BOBL* (ticker: OE1) - Duración a 01/10/2025: 5.44
- Futuros sobre el *BUND* (ticker: RX1) - Duración a 01/10/2025: 10

*Contract size* en todos los casos: 100,000 euros

Investiga sobre estos instrumentos antes de tomar la decisión. Razona tu elección del instrumento y el número de contratos que has decidido comprar/vender.

- ¿Qué pasaría si comprásemos/vendiésemos 100 futuros?
- ¿Se te ocurre algún otro instrumento con el que cubrir la sensibilidad a los tipos de interés de la cartera?

</div>

### <font color=#336699>8. Cobertura crédito</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Utiliza alguno de los siguientes instrumentos de los que te hemos dado para cubrir el riesgo de crédito de la cartera que has construido según el mandato. Asume una inversión en la cartera de 10 millones:

- ITRAXX Main (ticker: ITRX EUR CDSI GEN 5Y Corp)
- ITRAXX XOVER (ticker: ITRX XOVER CDSI GEN 5Y Corp)

Estos índices cotizan en forma de spread, en puntos básicos. La sensibilidad del valor del swap (CDS) la vamos a asumir en 4,500€ al punto básico asumiendo una inversión de 10 millones.

Investiga sobre estos instrumentos antes de tomar la decisión. Razona tu elección del instrumento y el nominal que has decidido comprar/vender.

- ¿Tiene sentido plantear esta cobertura total?
- ¿Con qué otros instrumentos podrías cubrir el riesgo de crédito?

</div>

### <font color=#336699>9. Estrategia propia</font>

<style>.gray {background-color: #595959}

</style><div class="gray">

❕💬 Plantea tu propia estrategia con la información que tienes. Puede ser una estrategia direccional, de valor relativo, que hayas visto o no en clase; pero siempre razonando tu planteamiento.

</div>